In [1]:
import dataflowkit.datasets as D
import dataflowkit.graphs as G
import dataflowkit.recipes as R
from dataflowkit.utils.print_time import print_time, print_end
from multiprocessing import Process, Queue, cpu_count
import threading, numpy as np

In [2]:
class MemoryRecipe(R.BaseRecipe):
    def __init__(self):
        print('MemoryRecipe initialized')
        
    # Must be executed in singleton, maybe heavy memory demanding
    def execute(self, ins, outs):
        # Takes ~ 0.093899s
        for i in np.arange(1e6):
            pass

In [3]:
class CPURecipe(R.BaseRecipe):
    # Hevay calcuation process
    def execute(self, ins, outs):
        # Takes ~ 0.823933s
        for i in np.arange(1e7):
            pass

In [4]:
def process_job(file_queue, mrf, master_index):
    # declare recipes
    dummy_recipe1 = mrf.create(MemoryRecipe, no_slaves=1)
    dummy_recipe2 = CPURecipe()
        
    while not file_queue.empty():
        file = file_queue.get()
        print_time('master_index: ' + str(master_index) + ' file: ' + str(file))
        
        # declare datasets
        dataset1 = D.InMemory()
        dataset2 = D.InMemory()
        dataset3 = D.InMemory()
        
        # declare graph
        dummy_recipe1.execute(ins=dict(dataset1=dataset1), outs=dict(dataset2=dataset2))
        dummy_recipe2.execute(ins=dict(dataset2=dataset2), outs=dict(dataset3=dataset3))
        
        print_end('master_index: ' + str(master_index) + ' file: ' + str(file))

#### Declare the files be process

In [5]:
file_queue = Queue()
files = np.arange(20)
for file in files:
    file_queue.put(file)

#### MasterRecipeFactory must be initialized before opeining the processes

In [6]:
no_master = 4
mrf = G.MasterRecipeFactory.get_inatance()

#### Declare the processes

In [7]:
processes = [Process(target=process_job, args=(file_queue, mrf, master_index)) 
             for master_index in np.arange(no_master)]

#### Execute the processes

In [8]:
for p in processes:
    p.start()

MemoryRecipe initialized
master_index: 0 file: 0 2.153474
master_index: 2 file: 2 2.451004
master_index: 1 file: 1 2.487954
master_index: 3 file: 3 2.52214
master_index: 0 file: 4 1.380044
master_index: 2 file: 5 1.34168
master_index: 1 file: 6 1.358464
master_index: 3 file: 7 1.470402
master_index: 0 file: 8 1.403856
master_index: 2 file: 9 1.336174
master_index: 1 file: 10 1.339905
master_index: 3 file: 11 1.377648
master_index: 0 file: 12 1.49676
master_index: 2 file: 13 1.712059
master_index: 1 file: 14 1.744367
master_index: 3 file: 15 1.752966
master_index: 0 file: 16 1.510662
master_index: 2 file: 17 1.32906
master_index: 1 file: 18 1.305392
master_index: 3 file: 19 1.133146


#### Terminate processes

In [9]:
[p.terminate() for p in processes]

[None, None, None, None]